In [1]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import MultiDiscrete, Discrete

In [2]:
print(gym.__version__)

0.28.1


In [7]:
import random 

# Creating a Custom Environment



In [51]:
# make an environment
import random

class SchedulerEnvironment(gym.Env):
    def __init__(self, scenario_one = True, terminate_num = 200, mean_delay_one = 6, mean_delay_two = 4):
        self.scenario_one = scenario_one
        self.mean_delay_one = mean_delay_one
        self.mean_delay_two = mean_delay_two


        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 

        self.incoming_packets = dict(
            {
                0: self.queue_one_incoming_packet,
                1: self.queue_two_incoming_packet,
                2: self.queue_best_effort_incoming_packet

            }
            )


        self.step_counter = 0 # i.e. timeslots
        self.terminate_num = terminate_num

        # maximum size of the queue
        self.max_queue_size = 10
        
        # can adjust size of max delay in timeslots after initial observations
        MAX_DELAY = 100

        queue_sample = np.arange(self.max_queue_size)
        for i in range(self.max_queue_size):
            queue_sample[i] = MAX_DELAY

        self.observation_space = spaces.Dict( 
            {
                
                "queues": MultiDiscrete(queue_sample), 
                                
#                 boolean for timeslot delay during switch 
#                 zero for no delay, one for delay. 
                "switchCounter": Discrete(2)
                
            }
        )

        self.queue_one_delay = [] 

        self.queue_two_delay = []

        self.queue_best_effort_delay = [] 

        self.queue_delays = dict(
            {
                0: self.queue_one_delay,
                1: self.queue_two_delay,
                2: self.queue_best_effort_delay
            }
        )

        self.queue_one_removed = 0
        self.queue_two_removed = 0
        self.queue_best_effort_removed = 0

        self.removed_packets = dict(
            {
                0: self.queue_one_removed,
                1: self.queue_two_removed,
                2: self.queue_best_effort_removed
            }
        )

        self.current_state = None
        
        self.action_space = spaces.Discrete(3)
        
        self._action_to_queue = {
                0: self.queue_delays[0],
                1: self.queue_delays[1],
                2: self.queue_delays[2]
            }

    def _increment_delay(self):
        for i in range(len(self.queue_delays)):    
            for j in range(len(self.queue_delays[i])):
                self.queue_delays[i][j] += 1

            
    # def _add_delay_to_queue(self):
    #     queues = self.observation_space["queue_delay"]
    #     # this will give the index of the queue that had the packet selected
    #     selected_queue = self.action_space.sample() 
    #     for q in range(len(queues)):
    #         if (selected_queue != q):
    #             queues[q] += 1
    #         else:
    #             queues[q] = 0

    def _can_add_packets(self, packet_arrival, action ):
        # increment before evaluating arrival. 
        self.incoming_packets[action] += packet_arrival
        packet_arrived = self.incoming_packets[action] >= 1.0
        not_max_size = len(self.queue_delays[action]) < self.max_queue_size

        if packet_arrived and not_max_size:
            self.incoming_packets[action] -= 1.0
            self.queue_delays[action].append(0)                     

    def _add_packets(self):
        
        PACKET_ARRIVAL_ONE = 0.3
        PACKET_ARRIVAL_TWO = 0.25
        PACKET_ARRIVAL_BEST_EFFORT = 0.4
        
        self._can_add_packets(PACKET_ARRIVAL_ONE, 0)
        self._can_add_packets(PACKET_ARRIVAL_TWO, 1)
        self._can_add_packets(PACKET_ARRIVAL_BEST_EFFORT, 2)

        return
     
    def _modify_states(self):

        # will not calculate delay for freshly added packets. 
        self._increment_delay()
        self._add_packets()
        return 
    
    def _get_obs(self):
        return { 
            "queues": self.queue_delays
            }
    
    def _calculate_avg_delay(self, queue):
        queue_size = len(queue)
        if (queue_size == 0):
            return 0
        
        # sum
        sum = 0
        for i in range(queue_size):
            sum += queue[i]
        
        # avg
        avg = sum / queue_size
        return avg

    def _get_info(self):

        avg_one = self._calculate_avg_delay(self.queue_delays[0])
        avg_two = self._calculate_avg_delay(self.queue_delays[1])
        avg_best = self._calculate_avg_delay(self.queue_delays[2])

        # return average delays for each queue. 
        avg_mean_all_queues = [avg_one, avg_two, avg_best]
        
        # return the number of packets removed in each queue. 
        
        packets_removed = [self.removed_packets[0], self.removed_packets[1], self.removed_packets[2]]

        # return the queue num.
        return [avg_mean_all_queues, packets_removed, self.step_counter + 1]
 
    def _initialise_delays(self, queue):
        # give a random delay between 0 and 5
        size_of_queue = len(queue)
        for i in range(size_of_queue):
            queue[i] = random.randint(0, 5)

    def _set_all_zero(self):
        for i in range((3)):
            self.queue_delays[i].clear()
            self.removed_packets[i] = 0

        self.step_counter = 0
        self.current_state = None
        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 
    
    def reset(self, seed=None, options=None):

        # reset everything 
        self._set_all_zero()

        # set the size of the queue. 
        self.queue_delays[0] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[1] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[2] = self.observation_space["queues"].sample().tolist()

        # set seed before randomising 
        random.seed(seed)

        self._initialise_delays(self.queue_delays[0])
        self._initialise_delays(self.queue_delays[1])
        self._initialise_delays(self.queue_delays[2])

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def _reward_function(self, action, packet_delay):
        reward = 0

        mean_delays = dict(
            {
                0: self.mean_delay_one,
                1: self.mean_delay_two
            }
        )
        
        MARGIN_OF_DELAY = 1

        # determine reward (with packet_delay)
#         (mean_delays[action] > packet_delay + MARGIN_OF_DELAY)
        if (action == 2):
        
            # reward taking the best effort queue when mean delays are adjusted
            # for priority queues.
            avg_first_queue = self._calculate_avg_delay(self.queue_delays[0])
            within_margin_first_queue = ( avg_first_queue < mean_delays[0] + MARGIN_OF_DELAY) and ( avg_first_queue > mean_delays[0] - MARGIN_OF_DELAY )
            
            avg_second_queue = self._calculate_avg_delay(self.queue_delays[1])
            within_margin_second_queue = ( avg_second_queue < mean_delays[1] + MARGIN_OF_DELAY) and ( avg_second_queue > mean_delays[1] - MARGIN_OF_DELAY )
            
            if (within_margin_first_queue and within_margin_second_queue):
                reward = 1

            # Punish for not prioritising. 
            else:
                reward = -1

        else:

            # packet delay > mean delay by the margin
            if (mean_delays[action] + MARGIN_OF_DELAY < packet_delay):
                # encourage minimising the delay for this queue.
                reward = 0

            # packet delay < mean delay by the margin
            elif (mean_delays[action] - MARGIN_OF_DELAY > packet_delay):
                # discourage minimising the delay too much.
                reward = -1

            # packet delay within margin, don't perform again. 
            else:
                reward = -1

        return reward 
    
    def _retrieve_packet(self, action):
        # retrieve the packet. 
        packet_delay = self.queue_delays[action].pop(0)
        self.removed_packets[action] += 1


        return packet_delay

    def step(self, action):
        packet_delay = 0
        # reward will be -1.
        if len(self.queue_delays[action]) == 0:
            None

        else:
            if self.scenario_one:
                packet_delay = self._retrieve_packet(action)
    
            else: 
                if (action == self.current_state):
                    packet_delay = self._retrieve_packet(action)


        # perform queue switch given the conditions.
        if not self.scenario_one:
            if (action != self.current_state):
                self.current_state = action
        
        reward = self._reward_function(action, packet_delay)
                
        self._modify_states()

        info = self._get_info()

        terminated = False
        if self.step_counter + 1 == self.terminate_num:
            terminated = True
        self.step_counter += 1

        # observation made after modifying states. 
        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


In [52]:
env = SchedulerEnvironment(terminate_num = 50) 

In [53]:
def print_info(info):
    print("----------")
    print("current step =", info[2])
    print("-----")
    print("Average delay of first queue:", info[0][0])
    print("Average delay of second queue:", info[0][1])
    print("Average delay of best-effort queue:", info[0][2])
    print("-----")
    print("Packets removed from first queue:", info[1][0])
    print("Packets removed from second queue:", info[1][1])
    print("Packets removed from best-effort queue:", info[1][2])
    

In [54]:
def print_observation(observation):
    print("----------")
    print("First queue:", observation["queues"][0])
    print("Second queue:", observation["queues"][1])
    print("Best effort queue:", observation["queues"][2])
    

In [55]:
env.action_space.sample()

2

In [57]:
obs, info = env.reset()
print_observation(obs)
print_info(info)

----------
First queue: [4, 0, 5, 2, 4, 3, 0, 4, 3, 5]
Second queue: [1, 0, 4, 5, 2, 1, 5, 2, 5, 2]
Best effort queue: [3, 5, 3, 1, 4, 5, 1, 1, 0, 1]
----------
current step = 1
-----
Average delay of first queue: 3.0
Average delay of second queue: 2.7
Average delay of best-effort queue: 2.4
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 0


In [61]:
def random_scheduler(seed = None):
    terminated = False 
    truncated = False

    # reset seeds the randomiser
    env.reset(seed)
    score = 0

    while not terminated:  
    #     random 
        action = random.randint(0, 2)
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward
        print_observation(obs)
        print_info(info)
    print(score)

In [63]:
random_scheduler(0)

----------
First queue: [4, 4, 1, 3, 5, 4, 4, 3, 4, 3]
Second queue: [5, 2, 5, 2, 3, 2, 1, 5, 3, 5]
Best effort queue: [5, 2, 3, 1, 6, 1, 6, 3, 4, 0]
----------
current step = 1
-----
Average delay of first queue: 3.5
Average delay of second queue: 3.3
Average delay of best-effort queue: 3.1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [5, 2, 4, 6, 5, 5, 4, 5, 4]
Second queue: [6, 3, 6, 3, 4, 3, 2, 6, 4, 6]
Best effort queue: [6, 3, 4, 2, 7, 2, 7, 4, 5, 1]
----------
current step = 2
-----
Average delay of first queue: 4.444444444444445
Average delay of second queue: 4.3
Average delay of best-effort queue: 4.1
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [6, 3, 5, 7, 6, 6, 5, 6, 5]
Second queue: [4, 7, 4, 5, 4, 3, 7, 5, 7, 0]
Best effort queue: [7, 4, 5, 3, 8, 3, 8, 5, 6, 2]
----------
cur

Average delay of best-effort queue: 9.9
-----
Packets removed from first queue: 6
Packets removed from second queue: 7
Packets removed from best-effort queue: 10
----------
First queue: [27, 26, 27, 26, 20, 16, 10, 8, 7, 1]
Second queue: [28, 26, 28, 21, 18, 14, 10, 6, 2]
Best effort queue: [18, 17, 15, 12, 9, 6, 4, 3, 2, 0]
----------
current step = 24
-----
Average delay of first queue: 16.8
Average delay of second queue: 17.0
Average delay of best-effort queue: 8.6
-----
Packets removed from first queue: 6
Packets removed from second queue: 7
Packets removed from best-effort queue: 11
----------
First queue: [28, 27, 28, 27, 21, 17, 11, 9, 8, 2]
Second queue: [27, 29, 22, 19, 15, 11, 7, 3]
Best effort queue: [19, 18, 16, 13, 10, 7, 5, 4, 3, 1]
----------
current step = 25
-----
Average delay of first queue: 17.8
Average delay of second queue: 16.625
Average delay of best-effort queue: 9.6
-----
Packets removed from first queue: 6
Packets removed from second queue: 8
Packets removed 

In [64]:
random_scheduler(1)

----------
First queue: [2, 5, 1, 3, 1, 4, 4, 4, 6, 4]
Second queue: [2, 1, 4, 1, 4, 4, 5, 1, 6, 4]
Best effort queue: [6, 2, 5, 1, 3, 1, 1, 1, 6, 0]
----------
current step = 1
-----
Average delay of first queue: 3.4
Average delay of second queue: 3.2
Average delay of best-effort queue: 2.6
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [6, 2, 4, 2, 5, 5, 5, 7, 5]
Second queue: [3, 2, 5, 2, 5, 5, 6, 2, 7, 5]
Best effort queue: [7, 3, 6, 2, 4, 2, 2, 2, 7, 1]
----------
current step = 2
-----
Average delay of first queue: 4.555555555555555
Average delay of second queue: 4.2
Average delay of best-effort queue: 3.6
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [7, 3, 5, 3, 6, 6, 6, 8, 6]
Second queue: [3, 6, 3, 6, 6, 7, 3, 8, 6]
Best effort queue: [8, 4, 7, 3, 5, 3, 3, 3, 8, 2]
----------
curren

----------
First queue: [33, 30, 26, 23, 20, 16, 13, 10, 3, 1]
Second queue: [22, 18, 14, 10, 6, 2]
Best effort queue: [19, 17, 16, 15, 13, 12, 8, 6, 4, 0]
----------
current step = 50
-----
Average delay of first queue: 17.5
Average delay of second queue: 12.0
Average delay of best-effort queue: 11.0
-----
Packets removed from first queue: 14
Packets removed from second queue: 16
Packets removed from best-effort queue: 20
-24


In [65]:
random_scheduler(2)

----------
First queue: [1, 1, 1, 3, 2, 6, 6, 3, 3, 5]
Second queue: [5, 1, 5, 6, 2, 4, 6, 4, 6]
Best effort queue: [5, 3, 5, 4, 5, 3, 1, 1, 3, 4]
----------
current step = 1
-----
Average delay of first queue: 3.1
Average delay of second queue: 4.333333333333333
Average delay of best-effort queue: 3.4
-----
Packets removed from first queue: 0
Packets removed from second queue: 1
Packets removed from best-effort queue: 0
----------
First queue: [2, 2, 2, 4, 3, 7, 7, 4, 4, 6]
Second queue: [2, 6, 7, 3, 5, 7, 5, 7, 0]
Best effort queue: [6, 4, 6, 5, 6, 4, 2, 2, 4, 5]
----------
current step = 2
-----
Average delay of first queue: 4.1
Average delay of second queue: 4.666666666666667
Average delay of best-effort queue: 4.4
-----
Packets removed from first queue: 0
Packets removed from second queue: 2
Packets removed from best-effort queue: 0
----------
First queue: [3, 3, 3, 5, 4, 8, 8, 5, 5, 7]
Second queue: [7, 8, 4, 6, 8, 6, 8, 1]
Best effort queue: [7, 5, 7, 6, 7, 5, 3, 3, 5, 6]
------

----------
current step = 38
-----
Average delay of first queue: 18.4
Average delay of second queue: 12.0
Average delay of best-effort queue: 10.7
-----
Packets removed from first queue: 11
Packets removed from second queue: 13
Packets removed from best-effort queue: 14
----------
First queue: [32, 31, 29, 25, 22, 19, 15, 12, 8, 1]
Second queue: [21, 17, 13, 9, 5, 1]
Best effort queue: [21, 20, 19, 15, 14, 12, 6, 5, 3, 2]
----------
current step = 39
-----
Average delay of first queue: 19.4
Average delay of second queue: 11.0
Average delay of best-effort queue: 11.7
-----
Packets removed from first queue: 11
Packets removed from second queue: 14
Packets removed from best-effort queue: 14
----------
First queue: [33, 32, 30, 26, 23, 20, 16, 13, 9, 2]
Second queue: [18, 14, 10, 6, 2]
Best effort queue: [22, 21, 20, 16, 15, 13, 7, 6, 4, 3]
----------
current step = 40
-----
Average delay of first queue: 20.4
Average delay of second queue: 10.0
Average delay of best-effort queue: 12.7
----

In [16]:
def is_empty(queue):
    if (queue == []):
        return True
    else:
        return False

In [17]:
# FIFO
def fifo_action(obs):
    first_queue = obs["queues"][0]
    second_queue = obs["queues"][1]
    best_effort_queue = obs["queues"][2]
    
    first_queue_element = None
    second_queue_element = None
    best_effort_queue_element = None
    
    if (not is_empty(first_queue)):
        first_queue_element = first_queue[0]
    
    if (not is_empty(second_queue_element)):
        second_queue_element = second_queue[0]
    
    if (not is_empty(best_effort_queue)):
        best_effort_queue_element = best_effort_queue[0]
    
    largest_delay = max(first_queue_element, second_queue_element, best_effort_queue_element)
    
    action = 0
    if (largest_delay == best_effort_queue_element):
        action = 2
    elif (largest_delay == second_queue_element):
        action = 1
    
    return action

In [66]:
def fifo_scheduler(seed = None):
    terminated = False 
    truncated = False
    obs, info = env.reset(seed)
    score = 0
    while not terminated: 

    #     fifo
        action = fifo_action(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        score += reward
        print_observation(obs)
        print_info(info)
    print(score)

In [67]:
fifo_scheduler(0)

----------
First queue: [4, 4, 1, 3, 5, 4, 4, 3, 4, 3]
Second queue: [5, 2, 5, 2, 3, 2, 1, 5, 3, 5]
Best effort queue: [5, 2, 3, 1, 6, 1, 6, 3, 4, 0]
----------
current step = 1
-----
Average delay of first queue: 3.5
Average delay of second queue: 3.3
Average delay of best-effort queue: 3.1
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [5, 5, 2, 4, 6, 5, 5, 4, 5, 4]
Second queue: [6, 3, 6, 3, 4, 3, 2, 6, 4, 6]
Best effort queue: [3, 4, 2, 7, 2, 7, 4, 5, 1, 0]
----------
current step = 2
-----
Average delay of first queue: 4.5
Average delay of second queue: 4.3
Average delay of best-effort queue: 3.5
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 2
----------
First queue: [6, 6, 3, 5, 7, 6, 6, 5, 6, 5]
Second queue: [4, 7, 4, 5, 4, 3, 7, 5, 7]
Best effort queue: [4, 5, 3, 8, 3, 8, 5, 6, 2, 1]
----------
current step =

Average delay of second queue: 11.0
Average delay of best-effort queue: 12.1
-----
Packets removed from first queue: 3
Packets removed from second queue: 10
Packets removed from best-effort queue: 10
----------
First queue: [28, 27, 27, 26, 27, 26, 20, 19, 1, 0]
Second queue: [20, 16, 12, 8, 4, 0]
Best effort queue: [23, 22, 18, 17, 11, 10, 9, 8, 7, 6]
----------
current step = 24
-----
Average delay of first queue: 20.1
Average delay of second queue: 10.0
Average delay of best-effort queue: 13.1
-----
Packets removed from first queue: 4
Packets removed from second queue: 10
Packets removed from best-effort queue: 10
----------
First queue: [28, 28, 27, 28, 27, 21, 20, 2, 1, 0]
Second queue: [21, 17, 13, 9, 5, 1]
Best effort queue: [24, 23, 19, 18, 12, 11, 10, 9, 8, 7]
----------
current step = 25
-----
Average delay of first queue: 18.2
Average delay of second queue: 11.0
Average delay of best-effort queue: 14.1
-----
Packets removed from first queue: 5
Packets removed from second que

In [ ]:
# predefine a mean delay for first queue, second queue and best effort queue. 
MEAN_DELAY_ONE = 4
MEAN_DELAY_TWO = 6
MEAN_DELAY_THREE = 10